## 7. File-kezelés, kivételkezelés

### Fájlkezelés

Ahhoz, hogy egy fájl tartalmát egy python programmal olvashassuk, meg kell nyitnunk az __open__ paranccsal. Az __open__ parancs első paramétere a fájl elérési útja, a második paraméter a megnyitás módja, ez olvasás esetén "r" (az angol _read_ rövidítése). Az _open_ függvény egy fájl objektumot ad vissza, ezt tárolhatjuk egy változóban, amiből aztán kiolvashatjuk a fájl tartalmát:

In [2]:
f = open('data/movies.tsv', 'r', encoding="utf-8")

A fájl tartalmát beolvashatjuk egy listába, amelyben a fájl minden sora külön elem, erre szolgál a readlines parancs:

In [3]:
sorok = f.readlines()

In [7]:
sorok[0]

'Toy Story \t1995\tanimation,children,comedy\n'

A sorok végén egy újsor karakter ("\n") található, ezt a strip paranccsal lehet eltüntetni:

In [8]:
elso_sor = sorok[0].strip()

In [9]:
elso_sor

'Toy Story \t1995\tanimation,children,comedy'

Ebben az adatban az egyes mezőket TAB ("\t") választja el, a sort TAB-ok mentén szét kell vágnunk. Ehhez használhatjuk a már ismert __split__ parancsot, amelynek ezúttal argumentumot is átadunk: azt a karaktert, amely mentén vágni akarunk (ebben az esetben ez a "\t").

In [10]:
mezok = elso_sor.split("\t")

In [11]:
mezok

['Toy Story ', '1995', 'animation,children,comedy']

Hasonlóan kell szétvágnunk egy sor harmadik mezőjét, amelyben vesszők választják el az egyes műfajokat:

In [12]:
mufajok = mezok[2].split(',')

In [13]:
mufajok

['animation', 'children', 'comedy']

Ha egy fájlt már nem használunk többet, be kell zárni, hogy más programok is hozzáférhessenek:

In [18]:
f.close()

Ahhoz, hogy az egész adatot ilyen módon szétvágjuk, egy for ciklust kell írnunk, ami egy új listába helyezi el az adatot, immár megfelelően szétvágva:

In [5]:
def adatot_beolvas(fajl):
    f = open(fajl, 'r', encoding="utf-8")
    adat = []
    for line in f:
        mezok = line.strip().split('\t')
        mezok[0] = mezok[0].strip()
        mezok[2] = mezok[2].split(",")
        adat.append(mezok)
    f.close()
    return adat

Ebben a függvényben nem használtuk a readlines parancsot, a for ciklus magán a fájl objektumon is végig tud menni. Ilyenkor azonban a nyers adat nem lesz eltárolva, ha újra szükségünk van rá, újra ki kell olvasni a fájlból.

In [6]:
adat = adatot_beolvas("data/movies.tsv")

In [7]:
adat[:5]

[['Toy Story', '1995', ['animation', 'children', 'comedy']],
 ['GoldenEye', '1995', ['action', 'adventure', 'thriller']],
 ['Four Rooms', '1995', ['thriller']],
 ['Get Shorty', '1995', ['action', 'comedy', 'drama']],
 ['Copycat', '1995', ['crime', 'drama', 'thriller']]]

Ha egy fájlt írni szeretnénk, akkor write ("w") módban kell megnyitni. Vigyázat, ha így nyitunk meg egy már létező fájlt, az törli a tartalmát! Készítsünk most egy másik fájlt, ami csak a gyerekfilmeket tartalmazza:

In [8]:
f = open('data/movies_children.tsv', 'w', encoding="utf-8")
for film in adat:
    if 'children' not in film[2]:
        continue
    cim, ev, mufajok = film
    mufaj_mezo = ",".join(mufajok)
    mezok = [cim, ev, mufaj_mezo]
    sor = "\t".join(mezok)+'\n'
    f.write(sor)
f.close()

Nézzünk bele az új fájlba, hogy jól sikerült-e a kiírás:

In [9]:
uj_adat = adatot_beolvas('data/movies_children.tsv')

In [10]:
uj_adat[:5]

[['Toy Story', '1995', ['animation', 'children', 'comedy']],
 ['Babe', '1995', ['children', 'comedy', 'drama']],
 ['Free Willy 2: The Adventure Home',
  '1995',
  ['adventure', 'children', 'drama']],
 ['Santa Clause, The', '1994', ['children', 'comedy']],
 ['Lion King, The', '1994', ['animation', 'children', 'musical']]]

Persze a kiírást is jobb lenne egy függvénybe csomagolni, és különválasztani a szűréstől:

In [11]:
def adatot_kiir(adat, fajl):
    f = open(fajl, 'w')
    for film in adat:
        cim, ev, mufajok = film
        mufaj_mezo = ",".join(mufajok)
        mezok = [cim, ev, mufaj_mezo]
        sor = "\t".join(mezok)+'\n'
        f.write(sor)
    f.close()

A szűrés pedig egy független művelet, ezt tegyük be egy függvénybe, aminek tetszőleges műfajt át lehet adni:

In [12]:
def mufajra_szur(adat, mufaj):
    szurt_adat = []
    for film in adat:
        if mufaj in film[2]:
            szurt_adat.append(film)
    return szurt_adat

Próbáljuk ki ezt a függvényt egy másik műfajjal:

In [13]:
western_filmek = mufajra_szur(adat, "western")

In [14]:
western_filmek[:5]

[['Legends of the Fall', '1994', ['drama', 'romance', 'war', 'western']],
 ['Maverick', '1994', ['action', 'comedy', 'western']],
 ['Dances with Wolves', '1990', ['adventure', 'drama', 'western']],
 ['Good, The Bad and The Ugly, The', '1966', ['action', 'western']],
 ['Unforgiven', '1992', ['western']]]

Ezekből a függvényekből már könnyen írhatunk olyan függvényt is, ami beolvassa az adatot egy fájlból és kiírja egy szűrt változatát egy másik fájlba:

In [15]:
def fajlt_szur(mufaj):
    adat = adatot_beolvas('data/movies.tsv')
    szurt_adat = mufajra_szur(adat, mufaj)
    uj_fajl = "data/movies_"+mufaj+".tsv"
    adatot_kiir(szurt_adat, uj_fajl)    

In [16]:
fajlt_szur('comedy')

Nézzük meg, mit csináltunk:

In [17]:
vigjatekok = adatot_beolvas('data/movies_comedy.tsv')

In [18]:
vigjatekok[:5]

[['Toy Story', '1995', ['animation', 'children', 'comedy']],
 ['Get Shorty', '1995', ['action', 'comedy', 'drama']],
 ['Babe', '1995', ['children', 'comedy', 'drama']],
 ['Mighty Aphrodite', '1995', ['comedy']],
 ['French Twist', '1995', ['comedy', 'romance']]]

Az eddigiekből most már megépíthetjük azt a függvényt is, ami minden műfajra elvégzi a szűrést és szétszedi az adatot annyi fájlba, ahány különböző műfaj van az adatban. Ehhez már csak arra van szükség, hogy összeszedjük az összes műfajt, erre írunk még egy függvényt:

In [19]:
def osszes_mufajt_listaz(adat):
    mufajok = []
    for film in adat:
        for mufaj in film[2]:
            if mufaj not in mufajok:
                mufajok.append(mufaj)
    mufajok.sort()
    return mufajok        

In [20]:
mufajok = osszes_mufajt_listaz(adat)
mufajok

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'film_noir',
 'horror',
 'musical',
 'mystery',
 'romance',
 'sci_fi',
 'thriller',
 'unknown',
 'war',
 'western']

Ezután a teljes szétválogatás már nagyon egyszerű:

In [21]:
def mufajokat_szetvalogat(mufajok):
    for mufaj in mufajok:
        fajlt_szur(mufaj)

In [22]:
mufajokat_szetvalogat(mufajok)

Persze ez nem egy hatékony megoldás: 20-szor mentünk végig az adaton, pedig egyszer is elég lett volna, ilyet nagyobb adatnál nem engedhetünk meg magunknak.

### Kivételkezelés

A hibaüzeneteket, amiket már sokszor láttunk, a programozók __kivételeknek__ (exception) nevezik. Nézzük meg, miből épül fel egy kivétel, okozzunk egyet:

In [72]:
3/0

ZeroDivisionError: division by zero

Az utolsó sor megadja a kivétel típusát (jelen esetben _ZeroDivisionError_), ezt követi bármilyen további információ, amit tudhatunk az adott hiba okáról. Az utolsó sor előtti rész mutatja meg, hogy melyik sorok futtatásánál keletkezett a hiba. Nézzünk még egy példát:

In [73]:
int("pingvin")

ValueError: invalid literal for int() with base 10: 'pingvin'

Amikor egy szót próbálunk számmá konvertálni, _ValueError_ keletkezik, de ennél többet is tudunk, a : utáni rész részletezi a hibát, ti. hogy a "pingvin" nem értelmezhető számként (10-es számrendszerben).

Kivételkezelésnek azt hívjuk, amikor a kódot előre felkészítjük arra, hogy bizonyos típusú kivételeket "elviseljen", vagyis ha adott típusú hibákat okoz a futása, akkor ne álljon le, hanem valamit reagáljon. Az alábbi kód például beolvas egy számot, és ha nem tudja int-té konvertálni, akkor ezt írja ki.

In [74]:
while True:
    print("Írj be egy számot!")
    sztring = input()
    try:
        szam = int(sztring)
        break
    except ValueError:
        print("Ez nem szám!")

print("Ezt írtad be:", szam)

Írj be egy számot!
fkj
Ez nem szám!
Írj be egy számot!
sdfk
Ez nem szám!
Írj be egy számot!
pingvin
Ez nem szám!
Írj be egy számot!
23f
Ez nem szám!
Írj be egy számot!
15
Ezt írtad be: 15


A __try__ és __except__ szavak közötti blokkba kell írni azokat a parancsokat, amelyek során hibára számítunk. Az except után kell felsorolni azokat a hibatípusokat, amelyeket "el kell kapni", és ezt követi az a blokk, ami megadja, a hiba jelentkezése esetén milyen kód fusson le.

Így aztán megírhatunk egy fájl-beolvasó függvényt úgy, hogy ne okozzon problémát, ha a fájl egy-két sora hibás, vagy csak másmilyen, mint a többi. A "hp_characters.tsv" fájlban Harry Potter karakterek szerepelnek teljes névvel és leírással. Az alábbi függvény beolvassa a neveket egy listába, "vezetéknév, keresztnév" formában, de hibát okoz, ha egy név nem két szóból áll:

In [23]:
def neveket_beolvas(fajl):
    f = open(fajl, 'r', encoding="utf-8")
    nevek = []
    for line in f:
        nev, leiras = line.strip().split('\t')
        keresztnev, vezeteknev = nev.split()
        nevek.append(vezeteknev+", "+keresztnev)
    return nevek

In [24]:
neveket_beolvas('data/hp_characters.tsv')

ValueError: too many values to unpack (expected 2)

Kezeljük a hibát és írjuk ki azokat a neveket, amiket nem tudtunk így feldolgozni:

In [25]:
def neveket_beolvas(fajl):
    f = open(fajl, 'r', encoding="utf-8")
    nevek = []
    for line in f:
        nev, leiras = line.strip().split('\t')
        try:
            keresztnev, vezeteknev = nev.split()
            nevek.append(vezeteknev+", "+keresztnev)
        except ValueError:
            print("Hiba:", nev)
    return nevek        

In [26]:
neveket_beolvas('data/hp_characters.tsv')

Hiba: Regulus Arcturus Black
Hiba: Vincent Crabbe Sr.
Hiba: "Bartemius ""Barty"" Crouch Sr."
Hiba: "Bartemius ""Barty"" Crouch Jr."
Hiba: "Alastor ""Mad-Eye"" Moody"
Hiba: Tom Riddle Sr.
Hiba: Dolores Janes Umbridge
Hiba: Dobby
Hiba: Fluffy
Hiba: Hedwig
Hiba: Aragog
Hiba: Grawp


['Black, Sirius',
 'Brown, Lavender',
 'Chang, Cho',
 'Crabbe, Vincent',
 'Delacour, Fleur',
 'Diggory, Cedric',
 'Dumbledore, Alberforth',
 'Dumbledore, Albus',
 'Dursley, Dudley',
 'Dursley, Petunia',
 'Dursley, Vernon',
 'Filch, Argus',
 'Finnigan, Seamus',
 'Flamel, Nicolas',
 'Fudge, Cornelius',
 'Sr., Goyle',
 'Goyle, Gregory',
 'Granger, Hermione',
 'Hagrid, Rubeus',
 'Karkaroff, Igor',
 'Krum, Viktor',
 'Lestrange, Bellatrix',
 'Longbottom, Alice',
 'Longbottom, Frank',
 'Longbottom, Neville',
 'Lovegood, Luna',
 'Lovegood, Xenophilius',
 'Lupin, Remus',
 'Malfoy, Draco',
 'Malfoy, Lucius',
 'Malfoy, Narcissa',
 'Maxime, Olympe',
 'McGonagall, Minerva',
 'Pettigrew, Peter',
 'Potter, Harry',
 'Potter, James',
 'Potter, Lily',
 'Quirrell, Quirinus',
 'Riddle, Mary',
 'Voldemort, Lord',
 'Skeeter, Rita',
 'Snape, Severus',
 'Tonks, Nymphadora',
 'Weasley, Arthur',
 'Weasley, Bill',
 'Weasley, Charlie',
 'Weasley, Fred',
 'Weasley, George',
 'Weasley, Ginny',
 'Weasley, Molly',
 '

Próbáljuk ezeket a neveket is kezelni úgy, hogy minden név utolsó elemét tekintjük vezetéknévnek. Ay egyszavas neveket is külön kezeljük.

In [29]:
def neveket_beolvas(fajl):
    f = open(fajl, 'r', encoding="utf-8")
    nevek = []
    for line in f:
        nev, leiras = line.strip().split('\t')
        szavak = nev.strip('"').split()
        if len(szavak) == 1:
            nevek.append(szavak[0])
            continue
        try:
            keresztnev, vezeteknev = szavak
        except ValueError:
            vezeteknev = szavak[-1]
            keresztnev = " ".join(szavak[:-1])
        nevek.append(vezeteknev+", "+keresztnev)
    return nevek

In [30]:
neveket_beolvas('data/hp_characters.tsv')

['Black, Regulus Arcturus',
 'Black, Sirius',
 'Brown, Lavender',
 'Chang, Cho',
 'Sr., Vincent Crabbe',
 'Crabbe, Vincent',
 'Sr., Bartemius ""Barty"" Crouch',
 'Jr., Bartemius ""Barty"" Crouch',
 'Delacour, Fleur',
 'Diggory, Cedric',
 'Dumbledore, Alberforth',
 'Dumbledore, Albus',
 'Dursley, Dudley',
 'Dursley, Petunia',
 'Dursley, Vernon',
 'Filch, Argus',
 'Finnigan, Seamus',
 'Flamel, Nicolas',
 'Fudge, Cornelius',
 'Sr., Goyle',
 'Goyle, Gregory',
 'Granger, Hermione',
 'Hagrid, Rubeus',
 'Karkaroff, Igor',
 'Krum, Viktor',
 'Lestrange, Bellatrix',
 'Longbottom, Alice',
 'Longbottom, Frank',
 'Longbottom, Neville',
 'Lovegood, Luna',
 'Lovegood, Xenophilius',
 'Lupin, Remus',
 'Malfoy, Draco',
 'Malfoy, Lucius',
 'Malfoy, Narcissa',
 'Maxime, Olympe',
 'McGonagall, Minerva',
 'Moody, Alastor ""Mad-Eye""',
 'Pettigrew, Peter',
 'Potter, Harry',
 'Potter, James',
 'Potter, Lily',
 'Quirrell, Quirinus',
 'Sr., Tom Riddle',
 'Riddle, Mary',
 'Voldemort, Lord',
 'Skeeter, Rita',
 'S

### Egy összetettebb feladat

Végül nézzük meg, hogyan épül fel egy összetettebb program, ami fájlkezelést és kivételkezelést is tartalmaz. Az alábbi példában olyan alkalmazást írunk, ami beolvassa a filmes adatot, majd a felhasználót megkérdezi, hogy melyik évből, milyen műfajú filmekre kiváncsi, és csak ezeket listázza.

Először gondoljuk végig, milyen függvényekre lesz szükségünk:
- Adatbeolvasó függvény
- Év alapján válogató függvény
- Műfaj alapján válogató függvény
- Központi, a felhasználóval "kommunikáló" függvény

Rögtön azt is megtervezhetjük, melyik függvény milyen paramétereket fog kapni és mit fog visszaadni, ezalapján megírhatjuk mindegyik függvény "vázát":

In [108]:
def adatot_beolvas(fajlnev):
    adat = []
    # ide kerül maga a beolvasás
    return adat

In [109]:
def evre_szur(adat):
    szurt_adat = []
    # ide kerül a szűrés
    return szurt_adat

In [110]:
def mufajra_szur(adat):
    szurt_adat = []
    # ide kerül a szűrés
    return szurt_adat

In [113]:
def main():
    while True:
        print("Melyik évre vagy kiváncsi?")
        # ...
        print("És milyen műfajra?")
        # ...
        # ...

__7.1. FELADAT__ Írj olyan függvényt, ami sztringeket olvas be a felhasználótól, és kiírja őket egy fájlba, amíg a felhasználó azt nem írja, hogy "elég". A fájl neve legyen "user_input.txt". Minden sztring kerüljön új sorba a fájlban!

__7.2. FELADAT__ Írj olyan függvényt, ami a teljes Harry Potter adatot beolvassa egy listába (tehát minden listaelem egy kettő hosszúságú lista). Ellenőrizd is, hogy jól működik-e!

__7.3. FELADAT__ Írj olyan függvényt, ami a HP adatot kiírja az eredeti formátumban (TAB-bal elválasztott mezők).

__7.4. FELADAT__ Írj egy függvényt, ami a HP adatból kiválogatja a Weasley-ket, és visszaadja őket egy listában.

 __7.5. FELADAT__ Használd az előző két feladatban írt függvényeket, és írj olyan programot, ami beolvassa az adatot, kiválogatja a Weasley-ket, és kiírja őket egy "weasleyk.tsv" nevű fájlba.

__7.6 FELADAT (Pluszpontokért beadható! Határidő: 2016. november 9. 08:00)__

Írj olyan függvényt, aminek a Harry Potter-adatot tartalmazó fájl nevét kell átadni paraméterként, ő pedig megnyitja a fájlt, beolvassa az adatot, és a felhasználó vezetéknév alapján kereshet benne: valahányszor beír egy sztringet, megkeressük, hogy a táblázat hányadik sorában szerepel az adott sztring vezetéknévként, és írja ki az adott sorból a nevet és a leírást is. Segítség: szükség lesz egy listára, amiben csak a vezetéknevek vannak, és kivételkezelésre, ha nem található az adott sztring a listában!